In [ ]:
# imports

import os
import sys
import json
import importlib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools
import tools.omstools as omstools
import plotting.plottools as plottools

In [ ]:
# load occupancy from dqmio files

mes = ({
    'PXLayer_1': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1',
    'PXLayer_2': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2',
    'PXLayer_3': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3',
    'PXLayer_4': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_4',
})

eras = [
  #'Run2024A-v1', # only commissioning, no lumisections with physics flag set to True
  'Run2024B-v1',
  'Run2024C-v1',
  'Run2024D-v1',
  'Run2024E-v1',
  'Run2024E-v2',
  'Run2024F-v1',
  'Run2024G-v1',
  'Run2024H-v1',
  'Run2024I-v1',
  'Run2024I-v2',
  #'Run2024J-v1'  # pp reference run for heavy ion run; lower pileup and occupancy
]

datadir = '/eos/user/l/llambrec/dialstools-output'
year = '2024'
dataset = 'ZeroBias'
reco = 'PromptReco'

info = {}
for era in eras:
    info[era] = {}
    mainera, version = era.split('-')
    for melabel, mename in mes.items():
        f = f'{dataset}-{mainera}-{reco}-{version}-DQMIO-{mename}.parquet'
        f = os.path.join(datadir, f)
        df = iotools.read_parquet(f, columns=['run_number', 'ls_number', 'entries'])
        runs = df['run_number'].values
        lumis = df['ls_number'].values
        entries = df['entries'].values
        info[era][melabel] = {'runs': runs, 'lumis': lumis, 'entries': entries}

In [ ]:
# load oms json

omsjsons = {}
for era in eras:
    omsfile = 'omsdata/omsdata_{}.json'.format(era)
    with open(omsfile, 'r') as f:
        omsjsons[era] = json.load(f)

In [ ]:
# load golden json

# todo

In [ ]:
# define help functions for plotting

def occupancy_scatter_plot(info, omsattr, mes, eras,
                           colors='single', xaxlabel='auto', yaxlabel='Occupancy',
                           ymax=None, normalize = False,
                           physics_flag_filter = False,
                           json_filter = None,
                           dolegend = False):
    # define colors
    if colors=='single':
        colors = ['b']*len(eras)
    if colors=='perera':
        cmap = mpl.colormaps['viridis']
        colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    if colors=='perlumi':
        cmap = mpl.colormaps['jet']
        colorsequence = [cmap(i) for i in np.linspace(0., 1., num=len(info[eras[0]][list(mes.keys())[0]]['entries']), endpoint=True)]
        colors = [colorsequence]*len(eras)
    # initialize figure
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(18,6), squeeze=False)
    axs = axs.flatten()
    for meidx, melabel in enumerate(mes.keys()):
        ax = axs[meidx]
        for eraidx, era in enumerate(eras):
            print('Retrieving data for {}, {}'.format(melabel, era))
            runs = info[era][melabel]['runs']
            lumis = info[era][melabel]['lumis']
            entries = info[era][melabel]['entries']
            print('Found {} lumisections'.format(len(lumis)))
            # perform filtering
            if physics_flag_filter:
                physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, omsjsons[era], 'physics_flag')
                print('Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
                runs = runs[physics_flag_mask]
                lumis = lumis[physics_flag_mask]
                entries = entries[physics_flag_mask]
            if json_filter is not None:
                json_mask = jsonu.injson(runs, lumis, jsondict=json_filter)
                print('Json filter: {} / {} lumisections passing'.format(np.sum(json_mask), len(json_mask)))
                runs = runs[json_mask]
                lumis = lumis[json_mask]
                entries = entries[json_mask]
            nlumis_postfilter = len(lumis)
            values = omstools.find_oms_attr_for_lumisections(runs, lumis, omsjsons[era], omsattr)
            if normalize:
                entries = np.divide(entries, np.where(values>0, values, 1))
            ax.scatter(values, entries, s=1, alpha=0.25, color=colors[eraidx], label=era)
            if len(eras)==1: ax.text(0.05, 0.95, era + ', ' + melabel, va='top', transform=ax.transAxes, fontsize=12)
        if xaxlabel=='auto': xaxlabel = omsattr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        if ymax is not None: ax.set_ylim((0, ymax))
        if dolegend:
            ncols = len(eras)%4
            leg = ax.legend(loc='upper left', ncols=ncols)
            for handle in leg.legend_handles:
                handle._sizes = [30]
                handle.set_alpha(1)
        fig.subplots_adjust(bottom=-0.2, left=-0.2)
    return fig, axs

In [ ]:
# make plots for pileup

# per era
#for era in eras:
#    fig, axs = occupancy_scatter_plot(info, 'pileup', mes, [era], xaxlabel='Pileup', yaxlabel='Occupancy',
#                                      physics_flag_filter=True,
#                                      #json_filter=goldenjson
#                                     )
    
# eras together
fig, axs = occupancy_scatter_plot(info, 'pileup', mes, eras, colors='perera', xaxlabel='Pileup', yaxlabel='Occupancy',
                                  ymax=1e7, dolegend=True,
                                  physics_flag_filter=True,
                                  #json_filter=goldenjson
                                 )
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plot for pileup with normalized occupancy

fig, axs = occupancy_scatter_plot(info, 'pileup', mes, eras, colors='perera', xaxlabel='Pileup', yaxlabel='Occupancy (normalized)', normalize=True,
                                  ymax=3e5, dolegend=True,
                                  physics_flag_filter=True,
                                  #json_filter=goldenjson
                                 )
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plots for delivered lumi

# per era
#for era in eras:
#    fig, axs = occupancy_scatter_plot(info, 'delivered_lumi_per_lumisection', mes, [era], 
#                                      xaxlabel='Delivered luminosity per lumisection (per pb)', 
#                                      yaxlabel='Occupancy',
#                                      physics_flag_filter=True,
#                                      #json_filter=goldenjson
#                                     )
    
# eras together
fig, axs = occupancy_scatter_plot(info, 'delivered_lumi_per_lumisection', mes, eras,
                                  xaxlabel='Delivered luminosity per lumisection (per pb)', 
                                  yaxlabel='Occupancy',
                                  colors='perera',
                                  ymax=1.3e7,
                                  dolegend=True,
                                  physics_flag_filter=True,
                                  #json_filter=goldenjson
                                 )
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plots for recorded lumi

# per era
#for era in eras:
#    fig, axs = occupancy_scatter_plot(info, 'recorded_lumi_per_lumisection', mes, [era], 
#                                      xaxlabel='Recorded luminosity per lumisection (per pb)', 
#                                      yaxlabel='Occupancy',
#                                      physics_flag_filter=True,
#                                      #json_filter=goldenjson
#                                     )
    
# eras together
fig, axs = occupancy_scatter_plot(info, 'recorded_lumi_per_lumisection', mes, eras,
                                  xaxlabel='Recorded luminosity per lumisection (per pb)', 
                                  yaxlabel='Occupancy',
                                  colors='perera',
                                  ymax=1.3e7,
                                  dolegend=True,
                                  physics_flag_filter=True,
                                  #json_filter=goldenjson
                                 )
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)